In [89]:
from proteobench.modules.denovo.denovo_lfq_DDA_HCD import DDAHCDDeNovoModule
from proteobench.io.parsing.parse_settings import ParseSettingsBuilder
from proteobench.io.parsing.parse_denovo import load_input_file
from proteobench.score.denovo.denovoscores import DenovoScores
from proteobench.datapoint.denovo_datapoint import DenovoDatapoint
from proteobench.io.params.adanovo import extract_params as extract_params_adanovo
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from psm_utils import Peptidoform
import os

import numpy as np
from typing import List, Dict, Tuple

import seaborn as sns

from proteobench.io.params.adanovo import extract_params as extract_params_adanovo
from proteobench.io.params.casanovo import extract_params as extract_params_casanovo
from proteobench.io.params.deepnovo import extract_params as extract_params_deepnovo
from proteobench.io.params.instanovo import extract_params as extract_params_instanovo
from proteobench.io.params.pointnovo import extract_params as extract_params_pointnovo
from proteobench.io.params.pihelixnovo import (
    extract_params as extract_params_pihelixnovo,
)
from proteobench.io.params.piprimenovo import (
    extract_params as extract_params_piprimenovo,
)

In [90]:
root = '/home/sam/benchmarking/ProteoBench/test/data/denovo'
path_to_gt = '/home/sam/benchmarking/ProteoBench/proteobench/io/parsing/io_parse_settings/denovo/lfq/DDA/HCD/module_settings.csv'
module = DDAHCDDeNovoModule(
    token=''
)

test_cases = {
    'AdaNovo': os.path.join(root, 'results', 'adanovo', 'denovo_benchmark_test.adanovo.mztab'),
    'Casanovo': os.path.join(root, 'results', 'casanovo', 'denovo_benchmark_test.casanovo.mztab'),
    'DeepNovo': os.path.join(root, 'results', 'deepnovo', 'denovo_benchmark_test.deepnovo.tab'),
    'PepNet': os.path.join(root, 'results', 'pepnet', 'denovo_benchmark_test.pepnet.tsv'),
    'Pi-HelixNovo': os.path.join(root, 'results', 'pihelixnovo', 'denovo_benchmark_test.pihelixnovo.tsv'),
    'Pi-PrimeNovo': os.path.join(root, 'results', 'piprimenovo', 'denovo_benchmark_test.piprimenovo.tsv'),
}

test_cases_configs = {
    'AdaNovo': os.path.join(root, 'configs', 'adanovo', 'config.yaml'),
    'Casanovo': os.path.join(root, 'configs', 'casanovo', 'config.yaml'),
    'DeepNovo': os.path.join(root, 'configs', 'adanovo', 'config.yaml'),  # TODO: Update with correct parser if available
    'PepNet': os.path.join(root, 'configs', 'adanovo', 'config.yaml'),  # TODO: Update with correct parser if available
    'Pi-HelixNovo': os.path.join(root, 'configs', 'pihelixnovo', 'config.yaml'),
    'Pi-PrimeNovo': os.path.join(root, 'configs', 'piprimenovo', 'config.yaml'),
}

config_parsers = {
    "AdaNovo": extract_params_adanovo,
    "Casanovo": extract_params_casanovo,
    "DeepNovo": extract_params_adanovo,
    "PepNet": extract_params_adanovo,  # TODO: Update with correct parser if available
    "InstaNovo": extract_params_instanovo,
    "Pi-HelixNovo": extract_params_pihelixnovo,
    "Pi-PrimeNovo": extract_params_piprimenovo,
    "PointNovo": extract_params_pointnovo,
}

In [ ]:
# df = None
# path = test_cases['AdaNovo']
# tool = 'AdaNovo'
# input_file_loc=path
# input_format=tool
# user_input_=user_input.__dict__
# all_datapoints=df

# input_df = load_input_file(input_file_loc, input_format)
# parse_settings = ParseSettingsBuilder(
#     parse_settings_dir='/home/samva/Doctorate/Benchmarking/de_novo_module/ProteoBench/proteobench/io/parsing/io_parse_settings/denovo/lfq/DDA/HCD',
#     module_id='denovo_lfq_DDA_HCD'
# ).build_parser(input_format)
# standard_format = parse_settings.convert_to_standard_format(input_df)

# imds, df, input_df = module.benchmarking(
#     input_file_loc=path,
#     input_format=tool,
#     user_input=user_input.__dict__,
#     all_datapoints=df,
# )

In [91]:
dtps = None
for tool, path in test_cases.items():
    imds, dtps, input_df = module.benchmarking(
        input_file_loc=path,
        input_format=tool,
        user_input=config_parsers[tool](test_cases_configs[tool]).__dict__,
        all_datapoints=dtps,
    )

In [ ]:
# for tool, path in test_cases.items():
#     imds, df, input_df = module.benchmarking(
#         input_file_loc=path,
#         input_format=tool,
#         user_input=user_input.__dict__,
#         all_datapoints=df,
#     )

In [3]:
df = pd.read_pickle('df_test.pkl')

In [18]:
df = None
path = test_cases['DeepNovo']
tool = 'DeepNovo'
input_file_loc=path
input_format=tool
user_input_=user_input.__dict__
all_datapoints=df

input_df = load_input_file(input_file_loc, input_format)
parse_settings = ParseSettingsBuilder(
    parse_settings_dir='/home/samva/Doctorate/Benchmarking/de_novo_module/ProteoBench/proteobench/io/parsing/io_parse_settings/denovo/lfq/DDA/HCD',
    module_id='denovo_lfq_DDA_HCD'
).build_parser(input_format)
standard_format = parse_settings.convert_to_standard_format(input_df)

In [86]:
import json

jsons = []
for i, row in df.iterrows():
    with open(
        os.path.join(
            '/home/sam/benchmarking/submissions', 
            row['id']+'.json'
        ),
        'w',
        encoding='utf-8') as f:
        row_clean = row.where(pd.notnull(row), None)
        json.dump(row_clean.to_dict(), f, ensure_ascii=False, indent=2, allow_nan=False)

In [28]:
p = '/home/samva/Doctorate/Benchmarking/de_novo_module/Results_denovo_lfq_DDA_HCD/AdaNovo_20250619_105853.json'

pd.read_json(p, typ='series')

0    {"id":"AdaNovo_20250619_105853","software_name...
dtype: object

---